# Interpolation Testbed Notebook

Short notebook to test various interpolation strategies. 

In [1]:
# Standard library
import sys

# Data manipulation and analysis
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt

In [2]:
sys.path.insert(1, '../sealsml')

In [3]:
from baseline import create_meshgrid, remove_all_rows_with_val, GaussianProcessInterpolator, find_closest_values_with_indices, nanargmax_to_one, polar_to_cartesian

## Function

In [4]:
class GPModel():
    def __init__(self, mesh_dim=30, buffer=8, num_met_sensors =1):
        self.mesh_dim = mesh_dim
        self.buffer = buffer
        self.num_met_sensors = num_met_sensors

    def fit(self, x, y):
        print('fit does nothing')
        self.x_ = x
        self.y_ = y
       
        if not isinstance(self.x_, np.ndarray) or not isinstance(self.y_, np.ndarray):
            raise TypeError("Inputs must be NumPy arrays")
        
    def predict(self, x, y):
        '''

        '''
        self.encoder = x
        self.decoder = y

        # Create empty lists
        leak_loc = []
        sample_number = []
        print('encoder shape', np.shape(self.encoder))
        print('decoder shape', np.shape(self.decoder))
        # collapse on time
        median_time = np.median(self.encoder, axis=2) # this can be changed to 80th percentile or w/e

        # make xarray datasets
        dataset = xr.Dataset({"data": (["sample", "sensor", "variables", "mask"], median_time)})
                             
        decoder_dataset = xr.Dataset({"data": (["sample", "leak_locations","target_time", "variables", "mask"], self.decoder)})
    
        for i in dataset.sample:
            # pulling the information from the sensors, distance, azi_sin and azi_cos and ch4
            ref_dist = dataset.isel(mask = 0, sample=i).data.values[:,0]
            azi_sin  = dataset.isel(mask = 0, sample=i).data.values[:,1]
            azi_cos  = dataset.isel(mask = 0, sample=i).data.values[:,2]
            ch4_data = dataset.isel(mask = 0, sample=i).data.values[:,7]

            combined_sensor = np.column_stack((ref_dist, azi_sin, azi_cos, ch4_data))
            combined_sensor = combined_sensor[self.num_met_sensors:] # drops the met sensor
            drop_masked_sensor =  remove_all_rows_with_val(combined_sensor, -1)
            
            x_, y_ = polar_to_cartesian(drop_masked_sensor[:,0], drop_masked_sensor[:,1], drop_masked_sensor[:,2])

            ## Making the grid ## 
            # This was pulled out of the loop before, would be great to find a min/max x,y workflow
            x_new, y_new = create_meshgrid(x_, x_, buffer=self.buffer, grid_points=self.mesh_dim)
           
            ch4_data = drop_masked_sensor[:,3]
    
            # new mesh data points
            X_test = np.column_stack((x_new.ravel(), y_new.ravel()))

            X_train = np.column_stack((x_, y_))
            y_train = ch4_data

            ## Make the model
            gp_mo = GaussianProcessInterpolator(length_scale=10, n_restarts_optimizer=10, normalize_y=True) # this needs to be small to not barf
            gp_mo.fit(X_train, y_train)

            # Fit it - Interpolated Results
            reshaped_gp_results = gp_mo.predict(X_test, output_shape = (self.mesh_dim, self.mesh_dim))

            # Leak Locations
            ref_dist_leak = decoder_dataset.isel(mask = 0, target_time=0, sample=i).data.values[:,0]
            azi_sin_leak  = decoder_dataset.isel(mask = 0, target_time=0, sample=i).data.values[:,1]
            azi_cos_leak  = decoder_dataset.isel(mask = 0, target_time=0, sample=i).data.values[:,2]

            combined_leak_loc = np.column_stack((ref_dist_leak, azi_sin_leak, azi_cos_leak))
            drop_masked_leaks =  remove_all_rows_with_val(combined_leak_loc, -1)

            x_leaks, y_leaks = polar_to_cartesian(drop_masked_leaks[:,0], drop_masked_leaks[:,1], drop_masked_leaks[:,2])

            # Let's find the leak locations, and then mark that with a 1
            closest_values_x, indicies_x = find_closest_values_with_indices(x_leaks, x_new.diagonal())
            closest_values_y, indicies_y = find_closest_values_with_indices(y_leaks, y_new.diagonal())
            gp_    = reshaped_gp_results[indicies_x, indicies_y]
    
            # need to pad to 20 to match leak locations
            padded_array = np.pad(nanargmax_to_one(gp_), (0, decoder_dataset.leak_locations.size - len(nanargmax_to_one(gp_))), mode='constant')

            # append it
            sample_number.append(i) 

            leak_loc.append(padded_array)

        return np.asarray(leak_loc)  

## Loading in real data

Since none of these need to be 'trained', if it works on one dataset, it will work on all of them. 

In [5]:
data = '../data/training_data_SBL2m_Ug10_src10kg_b.0.nc'

In [6]:
ds = xr.open_dataset(data)
ds

<xarray.Dataset>
Dimensions:         (variable: 8, sample: 600, sensor: 10, time: 100, mask: 2,
                     pot_leak: 20, target_time: 1, sensor_loc: 3)
Coordinates:
  * variable        (variable) object 'ref_distance' 'ref_azi_sin' ... 'q_CH4'
  * sensor_loc      (sensor_loc) object 'xPos' 'yPos' 'zPos'
Dimensions without coordinates: sample, sensor, time, mask, pot_leak,
                                target_time
Data variables:
    encoder_input   (sample, sensor, time, variable, mask) float64 ...
    decoder_input   (sample, pot_leak, target_time, variable, mask) float64 ...
    target          (sample, pot_leak, target_time) float64 ...
    sensor_meta     (sample, sensor, sensor_loc) float64 ...
    leak_meta       (sample, pot_leak, sensor_loc) float64 ...
    met_sensor_loc  (sample, sensor_loc) float64 ...

## Testing new scikit-learn interface

In [7]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Filter out ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [8]:
model = GPModel(mesh_dim=60, buffer=4)

In [9]:
x = ds.encoder_input.values
y = ds.decoder_input.values

In [10]:
type(x)

numpy.ndarray

In [11]:
model.fit(x, y)

fit does nothing


In [12]:
dataset2 = model.predict(ds.encoder_input.values, ds.decoder_input.values)
dataset2

encoder shape (600, 10, 100, 8, 2)
decoder shape (600, 20, 1, 8, 2)


array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
type(dataset2)

numpy.ndarray

In [14]:
dataset2.shape

(600, 20)